<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt

Note to lecturers:  This notebook is designed to work best as a classic Jupyter Notebook with nbextensions 
* hide_input: to hide selected python cells particularly for just plotting
* RISE:  Interactive js slide presentations

# Introduction to Numerical Methods

## Outline

* Introduction and Motivation
* Some examples of Numerical Problems
* Overview of Course materials
* Course Logistics
* <font color=red>**Prerequisites and "Fluency"**</font>


## What are Numerical Methods?

* An extremely broad field -- much broader than this class

* Rough <font color=red>definition</font>:  analysis and application of algorithms to allow computers to solve problems in math, science, engineering...

* Strictly speaking, numerical methods don't require computers (e.g. Newton's method is ~ 17 century)

* But computers make things practical...Consider this class an introduction to computational math

## Why do we need numerical methods?

### Some problems have <font color=red>**no analytic solution** </font>
    
   Example:  find $f(x)=0$ for the 5th order polynomial
    
$$
    x^5 + 3x^2+ 2x + 3 = 0
$$

$$
    p(x) = x^5 + 3x^2+ 2x + 3 = 0
$$

In [ ]:
p = lambda x:  x**5 +3*x**2 +2*x +3

from scipy.optimize import brentq

x = numpy.linspace(-2,2,100)
x0 = brentq(p,-2,2)

plt.figure(figsize=(8,6))
plt.plot(x,p(x),linewidth=2)
plt.plot(x,numpy.zeros(x.shape),'k',linewidth=2)
plt.scatter(x0,p(x0),s=50,c='r')
plt.grid()
plt.xlabel('x',fontsize=16)
plt.ylabel('p(x)',fontsize=16)
plt.title('Root at x={}'.format(x0),fontsize=18)
plt.show()

The 5th order polynomial  $p(x) = x^5 + 3x^2+ 2x + 3 = 0$ actually has 5 complex roots

In [ ]:
c = [1,0,0,3,2,3]
roots = numpy.roots(c)
fig = plt.figure(figsize=(8,6))
plt.scatter(numpy.real(roots),numpy.imag(roots),s=50,c='r')
x=numpy.linspace(-2,2)
plt.plot(x,numpy.zeros(x.shape),'k',linewidth=2)
plt.plot(numpy.zeros(x.shape),x,'k',linewidth=2)

plt.xlabel('Re',fontsize=16)
plt.ylabel('Im',fontsize=16)
plt.grid()
plt.title('Complex roots of a 5th order polynomial')
plt.axis('square')
plt.xlim([-2,2])
plt.ylim([-2,2])
plt.show()

which require completely different numerical methods to find.

In general, the roots of most nonlinear systems,
    
$$
    \mathbf{F}(x,y,z,t) = \mathbf{0},
$$
    
cannot be solved directly.


### Some problems are too big to be done by hand. 

It may not possible to calculate an exact answer in a convenient amount of time. It may be that a system is over constrained and obtaining an answer that matches all requirements is not possible.

![Linear Regression](./images/linear_regression.png)

### Sometimes you actually want an answer 

(rather than show it exists or is unique)

* Example:  Solution of non-linear Dynamical systems

$$
    \frac{d\mathbf{u}}{dt} = \mathbf{F}(t,\mathbf{u})\quad \mathbf{u}(0)=\mathbf{u}_0
    $$

* Numerics <font color=red>**complement** </font> analytic methods, but don't replace them
    
* You need both to <font color=red> **understand** </font> your problems

## Mathematical Modeling
* Numerical methods are only a part of the bigger problem of <font color=red>Mathematical Modeling</font>

<div>
<img src="./images/mathModeling.png" width="500"/>
</div>



* Murphy was a modeler...

We usually have experience and insights into a system and have a good idea what the solutions should be. At the same time the numerics can help us build our intuition and complement analytical methods. The numerics can also be used to test ideas and extend our understanding of a system. In doing so we can update mathematical descriptions to account for new aspects.

The process of describing physical systems using mathematical expressions is often referred to as mathematical modeling. It is a process in which intuition, simulation, and analysis are used to build expressions that mimic general behaviours of physical systems. The general practice is to start with a basic description and refine it over many iterations. 

Each step in the process introduces error. In this course we will focus on the errors associated with numerical approximation. It is important, though, to be able to determine the difference between errors associated with simplifications associated with the model and the errors associated with the approximation of mathematical expressions.

## Why should I care?

*Because I have to be here for a requirement...*

*or I might actually need to solve something important...like can I ever retire?*

### An example: The Retirement Problem

$$A = \frac{P}{r} \left((1+r)^n - 1 \right)$$

$A$ is the total amount after n payments

$P$ is the incremental payment

$r$ is the interest rate per payment period

$n$ is the number of payments



Note that these can all be functions of $r$, $n$, and time

Plot the total rate of return over 20 years for saving \\$1000 per month at a yearly interest rate $r$

In [ ]:
def A(P, r, n):
    return P / r * ((1 + r)**n - 1)

P = 1000.
years = numpy.linspace(0,20,241)
n = 12*years
target = 1.e6

In [ ]:
fig = plt.figure(figsize=(8,6))
#fig.set_figwidth(fig.get_figwidth() * 2)

axes = fig.add_subplot(1, 1, 1)
for r in [0.02, 0.05, 0.08, 0.1, 0.12, 0.15]:
    axes.plot(years, A(P, r/12., n),label='r = {}'.format(r))
axes.plot(years, numpy.ones(years.shape) * target, 'k--',linewidth=2,label="Target")
axes.legend(loc='best')
axes.set_xlabel("Years",fontsize=18)
axes.set_ylabel("Annuity Value (Dollars)",fontsize=18)
axes.set_title("The Forward Problem $A(P,r,n)$",fontsize=20)
axes.grid()

In [ ]:
# find the root using scipy's brentq method
from scipy.optimize import brentq
n_max = n[-1]
f = lambda r : target - A(P,r/12., n_max)
r_target = brentq(f,0.1,0.15)

In [ ]:
fig = plt.figure(figsize=(8,6))
axes = fig.add_subplot(1, 1, 1)
r = numpy.linspace(1.e-6,0.15,100)
target = 1.e6
axes.plot(A(P,r/12.,n_max),r,linewidth=2)
axes.plot(numpy.ones(r.shape) * target, r,'k--',linewidth=2)
axes.scatter(A(P,r_target/12,n_max),r_target,s=100,c='r')
axes.set_xlabel('Annuity Value (Dollars)',fontsize=18)
axes.set_title('The Inverse Problem $r(A,P,n)$, $r\geq$ {:.3f}'.format(r_target),fontsize=20)
axes.grid()
plt.show()

This is a rootfinding problem for a function of a single variable

### Another Example:  Boat race (numerical quadrature of arc length)
Given a river with coordinates $(x,f(x))$ find the total length actually rowed over a given interval $x\in [0,X]$

Example:  a sinusoidal river $$f(x) = A \sin x$$

In [ ]:
A=.5
fig = plt.figure(figsize=(8,6))
x = numpy.linspace(0,10,100)
plt.plot(x,A*numpy.sin(x),'r',linewidth=2,)
plt.xlabel('distance (km)')
plt.grid()
plt.title('The Boat Race: $f(x)=A\sin(x)')
plt.show()

We need to calculate the function $f(x)$'s arc-length from $[0, X]$ e.g.

\begin{align}
    L(X) &= \int_0^{X} \sqrt{1 + |f'(x)|^2} dx\\
         &= \int_0^{X} \sqrt{1 + A^2\cos^2(x)} dx\\
\end{align}

In general the value of this integral cannot be solved analytically (the answer is actually an elliptic integral). We usually need to approximate the integral using a numerical quadrature.

### Non-Linear population growth

Lotka-Volterra Predator-Prey model: Rabbits v. Foxes

The variable $R$ represents the number of a prey animals in a population. The variable $F$ represents the number of a predators in a population. The interactions between the two can be approximated with the system of differential equations
$$
\begin{align}
    \frac{d R}{dt} &= R \cdot (a - bF)\\
    \frac{d F}{dt} &= F \cdot (-c + dR), 
\end{align}
$$

### Numerical Solutions 

$$
    R_0 = 10,\, F_0 = 5\quad  a,b,c,d = (1.5, 1, 3, 1)
$$


In [ ]:
# The Lotka Volterra predator prey problem using SciPy's 
#  ODE integrator solve_ivp

from scipy.integrate import solve_ivp

def lotkavolterra(t, u, a, b, c, d):
    r,f = u
    return [r*(a - b*f), f*(-c + d*r) ]

a,b,c,d = (1.5, 1, 3, 1)
f = lambda t,u : lotkavolterra(t,u,a,b,c,d)

time_span = [0, 15]
u_0 = [10, 5]
sol = solve_ivp(f, time_span , u_0, rtol=1.e-6, atol=1.e-9,dense_output = True)


In [ ]:
t = numpy.linspace(0, 15, 300)
z = sol.sol(t)

fig = plt.figure(figsize=(16,6))
axes = fig.add_subplot(1,2,1)
axes.plot(t,z[0],'r',label='rabbits')
axes.plot(t,z[1],'b',label='foxes')
axes.legend(loc='best',shadow=True)
axes.set_xlabel('Time',fontsize=16)
axes.set_ylabel('Population',fontsize=16)
axes.grid()
axes.set_title('Lotka-Volterra Predator Prey system',fontsize=18)

axes = fig.add_subplot(1,2,2)
axes.plot(z[0],z[1],linewidth=1)
axes.grid()
axes.set_xlabel('Rabbits',fontsize=16)
axes.set_ylabel('Foxes',fontsize=16)
axes.set_title('Lotka-Volterra phase portrait',fontsize=18)
plt.show()

#### Questions an ecologist or a mathematician might ask:
 - Where are the steady states?
 - Are the solutions to the system stable?
 - How do we solve the initial value problem?
 - How do we understand the non-linear dynamics?
 - How do we evaluate whether this is a good model?
 - What impacts do small changes to the different parameters have?

### Interpolation and Data Fitting

Finding trends in real data represented without a closed form (analytical form).

Sunspot counts

In [ ]:
data = numpy.loadtxt("./data/sunspot.dat")

fig = plt.figure(figsize=(10, 5))
fig.set_figwidth(fig.get_figwidth() * 2)

axes = fig.add_subplot(1, 2, 1)
axes.plot(data[:, 0], data[:, 1],linewidth=2)
axes.set_xlabel("Year",fontsize=16)
axes.set_ylabel("Number",fontsize=16)
axes.set_title("Number of Sunspots",fontsize=18)

axes = fig.add_subplot(1, 2, 2)
N = int(data.shape[0] / 2)
period = 1.0 / numpy.fft.fftfreq(data.shape[0])[1:N]
sunspot_freq = numpy.fft.fft(data[:, 1])[1:N]
freq = numpy.fft.fftfreq(data.shape[0])[1:N]
axes.plot(period, numpy.abs(sunspot_freq)**2,linewidth=2)
axes.set_xlabel("Years/Cycle",fontsize=16)
axes.set_ylabel("Power Spectrum",fontsize=16)
axes.set_title("Frequency of Sunspots",fontsize=18)
axes.set_xlim((0, 50))

plt.show()

## Why is this exciting?

Computers have revolutionized our ability to explore...

[Top 10 Algorithms of the 20th Century](http://www.siam.org/pdf/news/637.pdf?t=1&cn=ZmxleGlibGVfcmVjcw%3D%3D&refsrc=email&iid=658bdab6af614c83a8df1f8e02035eae&uid=755271476&nid=244+285282312)

...and there is more to come!

## Tools

### Computer Languages



#### C, C++, Fortran

##### Pros:
 - Performance and legacy computing codes available

##### Cons:
 - Syntax not optimized for casual programming
 - No interactive facilities
 - Difficult visualization, text processing, etc.
 

#### IDL, Matlab, Mathematica, etc.

##### Pros:
 - Interactive with easy visualization tools
 - Extensive scientific and engineering libraries available

##### Cons:
 - Costly and proprietary
 - Unpleasant for large-scale computing and non-mathematical tasks

#### Python
##### Features and Project Goals:
 - Python is free (BSD-like license) and highly portable (Windows, Mac OS X, Linux, etc.)
 - Interactive interpreter
 - Readability
 - Simple
 - Extensive documentation
 - Memory management is (mostly) transparent
 - Clean and object-oriented
 - Built-in types

##### Pros:
- Comprehensive standard library
 - Well-established 3rd-party packages (NumPy, SciPy, matplotlib, etc.)
 - Easily wraps existing legacy code in C, C++ and Fortran
 - Python mastery is marketable
 - Scalability
   - Interactive experimentation
   - Good tools for documenting code, and the resulting code tends to be easier to maintain compared to other interpreted languages.
   - Code can be one-line scripts or million-line projects
   - Used by novices and full-time professionals alike

##### Cons:
 - Can be slow
 - Packaging system is a bit crufty
 - Discipline is forced on the programmer by the design of the language. Can limit choices available for how to implement an algorithm. (The *python* way.)
 - Too many Monty Python jokes (not really a con)

### Version Control via `git`

`git` is version control system allowing a user to take "snapshots" of data.  These snapshots can then be compared, combined or recalled as needed.

Why version control (or `git`)?
 - Nearly ubiquitous in modern software engineering and hence an essential skill
 - Can save you from yourself
 - Greatly simplifies the way multiple people can work on and share a single project.
 - Steep learning curve so expose yourself often for maximal success
 
[Learn more!](https://git-scm.com/)

### Peer Review

**Why?**
In this class reviewing your peer's work can lead to
 - Better understanding of a problem
 - See alternative solutions to the same problem
 - Learn how to read other people's code
 - Hopefully learn some tips about your own coding style
 - Another skill highly used in practice
 - Demonstrate why readibility and discipline with respect to code matters.

## Infrastructure

### Python 3.x

### Jupyter Notebooks

The notebook environment gives us a convenient means for working with code while annotating it.  We will only cover the key concepts here and hope that you will explore on your own the environments.

[Documentation](https://jupyter.readthedocs.io/en/latest/)

#### Toolbar

 - Notebooks are modal, they have an edit mode (editing cells) and command mode.
 - Highly recommend taking the tour and checking out the help menu 

#### Content types
   - Markdown
   - LaTeX $x^2$
   - Python
   - NumPy, SciPy, and other packages

#### Jupyter Lab

### Obtaining the Notebooks

All notebooks are found on [github](http://github.com/mandli/intro-numerical-methods).

Highly recommend obtaining a github account if you do not already have one.  Will allow you start to become comfortable with `git`.

**Clone** the repository

`$> git clone git://github.com/mspieg/intro-numerical-methods`

**Pull** in new changes

`$> git pull`

**Push** new changes (you do not have permission to do this

`$> git push`

Also note that you can watch what changes were made and submit **issues** to the github project page if you find mistakes (PLEASE DO THIS!). It will be much easier to do so if you have your own account on github and work with your own clone. You can then submit a merge request to have your changes incorporated into other repositories.

### Installation

A few options
 1. Install on your own machine
 2. Use a cloud service

#### Your own machine

 - [Notebook Quick Start Guide](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/index.html)
 - [Anaconda](http://anaconda.com/downloads)
 - [Canopy](https://store.enthought.com/downloads/) 

Remember to grab the <font color=red> **Python 3.x** </font> versions!

#### The "cloud"

Instead of running things locally on your machine there are a number of cloud services that you are welcome to use in order to get everything running easily:
 - Columbia's CUIT Jupyter Hub server
 - [Google's Colaboratory](https://colab.research.google.com)
 - [Microsoft Azure Notebooks](https://notebooks.azure.com)
 - [CoCalc](https://cocalc.com/)

### BRING YOUR LAPTOP!

In class demos are better with your participation!